In [3]:
import numpy as np
import pandas as pd
import pylab as pl
import os

# Download Data and move to data folder

# Read data and visualise tables to ensure correct

In [4]:
# Read data into data frames
TaxiLookup = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/taxi-zone-lookup.csv")
UberApr14 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-apr14.csv")
UberMay14 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-may14.csv")
UberJun14 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-jun14.csv")
UberJul14 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-jul14.csv")
UberAug14 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-aug14.csv")
UberSep14 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-sep14.csv")
UberJanJun15 = pd.read_csv("/wingrdp/homedirs/adn323/ADS2017_adn323/Data/Project_Data/uber-raw-data-janjune-15.csv")

In [5]:
UberJanJun15.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90


In [30]:
TaxiLookup.head()

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [7]:
UberApr14.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


# Merge April to Septemebr 2014 data into 1 data frame

In [8]:
UberAprSep14 = pd.concat([UberApr14,UberMay14,UberJun14,UberJul14,UberAug14,UberSep14]).reset_index()
UberAprSep14.head()

,index,Date/Time,Lat,Lon,Base
0,0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [9]:
UberAprSep14["Base"].unique()

array(['B02512', 'B02598', 'B02617', 'B02682', 'B02764'], dtype=object)

In [10]:
UberAprSep14.shape

(4534327, 5)

In [11]:
UberJanJun15["Dispatching_base_num"].unique()

array(['B02617', 'B02598', 'B02682', 'B02764', 'B02512', 'B02765',
       'B02835', 'B02836'], dtype=object)

In [12]:
Bases = pd.DataFrame(UberAprSep14["Base"].unique())
Bases = Bases.rename(columns={0:"BasesAprSep14"})
Bases2 = pd.DataFrame(UberJanJun15["Dispatching_base_num"].unique())
Bases2 = Bases2.rename(columns={0:"BasesJanJun15"})

In [13]:
Bases

,BasesAprSep14
0,B02512
1,B02598
2,B02617
3,B02682
4,B02764


In [14]:
Bases2

,BasesJanJun15
0,B02617
1,B02598
2,B02682
3,B02764
4,B02512
5,B02765
6,B02835
7,B02836


# Pull in FHV TLC bases list to determine what are Uber bases

In [15]:
# Read in Taxi Bases Lists
TLCbasesComm = pd.read_excel("http://www.nyc.gov/html/tlc/downloads/excel/current_community_car_service_bases.xls")
TLCbasesBlack = pd.read_excel("http://www.nyc.gov/html/tlc/downloads/excel/current_black_car_bases.xls")
TLCbasesLux = pd.read_excel("http://www.nyc.gov/html/tlc/downloads/excel/current_luxury_limousine_bases.xls")

# Merge Taxi Bases list and drop irrelevant columns

In [16]:
TLCbasesComm.head(1)

,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE,STREET ADDRESS,CITY,ZIP CODE,TELEPHONE NUMBER,LICENSE TYPE DESC,TOTAL AFFILIATED,COMMUNITY BOARD,COUNCIL DISTRICT,NYPD PRECINCT,SHL ENDORSEMENT
0,B00008,T-D MAINTENANCE CORP,FOUR ONES CAR SERVICE,127-01 METROPOLITAN AVENUE,KEW GARDENS,11415,(718) 456-1111,TLC Licensed Base - Livery,30.0,5.0,30.0,104.0,N


In [17]:
TLCbasesBlack.head(1)

,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE,STREET ADDRESS,CITY,ZIP CODE,TELEPHONE NUMBER,LICENSE TYPE DESC,TOTAL AFFILIATED,COMMUNITY BOARD,COUNCIL DISTRICT,NYPD PRECINCT,SHL ENDORSEMENT
0,B00013,LOVE CORPORATE CAR INC,NaN,1440 39 STREET,BROOKLYN,11218,(718) 438-1100,TLC License Base - Black Car,116,12.0,39.0,66.0,N


In [18]:
TLCbasesLux.head(1)

,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATIVE NAME OF LICENSEE,STREET ADDRESS,CITY,ZIP CODE,LICENSE TYPE DESC,TELEPHONE NUMBER,TOTAL AFFILIATED,SHL ENDORSEMENT
0,B00001,LONDON TOWNCARS INC,NaN,40-14 23 STREET,LIC,11101,TLC Licensed Base - Luxury Limousine,(718) 786-9700,30.0,NaN


In [19]:
TLCbasesComm.columns

Index([u'LICENSEE NUMBER', u'NAME OF LICENSEE', u'ALTERNATE NAME OF LICENSEE',
       u'STREET ADDRESS', u'CITY', u'ZIP CODE', u'TELEPHONE NUMBER',
       u'LICENSE TYPE DESC', u'TOTAL AFFILIATED', u'COMMUNITY BOARD',
       u'COUNCIL DISTRICT', u'NYPD PRECINCT', u'SHL ENDORSEMENT'],
      dtype='object')

In [20]:
TLCbasesBlack.columns

Index([u'LICENSEE NUMBER', u'NAME OF LICENSEE', u'ALTERNATE NAME OF LICENSEE',
       u'STREET ADDRESS', u'CITY', u'ZIP CODE', u'TELEPHONE NUMBER',
       u'LICENSE TYPE DESC', u'TOTAL AFFILIATED', u'COMMUNITY BOARD',
       u'COUNCIL DISTRICT', u'NYPD PRECINCT', u'SHL ENDORSEMENT'],
      dtype='object')

In [21]:
TLCbasesLux.columns

Index([u'LICENSEE NUMBER', u'NAME OF LICENSEE',
       u'ALTERNATIVE NAME OF LICENSEE', u'STREET ADDRESS', u'CITY',
       u'ZIP CODE', u'LICENSE TYPE DESC', u'TELEPHONE NUMBER',
       u'TOTAL AFFILIATED', u'SHL ENDORSEMENT'],
      dtype='object')

In [22]:
# Drop Unnecessary Columns and rename ones that are inccorectly labeled
TLCbasesComm = TLCbasesComm[['LICENSEE NUMBER', 'NAME OF LICENSEE', 'ALTERNATE NAME OF LICENSEE']]
TLCbasesBlack = TLCbasesBlack[['LICENSEE NUMBER', 'NAME OF LICENSEE', 'ALTERNATE NAME OF LICENSEE']]
TLCbasesLux = TLCbasesLux[['LICENSEE NUMBER', 'NAME OF LICENSEE', 'ALTERNATIVE NAME OF LICENSEE']]

TLCbasesLux = TLCbasesLux.rename(columns={"ALTERNATIVE NAME OF LICENSEE":'ALTERNATE NAME OF LICENSEE'})

# Merge Taxi Bases list and drop irrelevant columns
TLCbases = pd.concat([TLCbasesComm,TLCbasesBlack,TLCbasesLux]).reset_index()

In [23]:
TLCbases.head()

,index,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE
0,0,B00008,T-D MAINTENANCE CORP,FOUR ONES CAR SERVICE
1,1,B00009,T-D MAINTENANCE,FOUR ONES CAR SERVICE
2,2,B00021,MEXICANA HIGH CLASS INC.,CAPITAL CAR SERVICE
3,3,B00029,COMMUNITY CAR SVC CORP,NaN
4,4,B00030,TREMONT DISPATCHING CORP,NaN


In [188]:
# The missing Taxi Base number does not exist in the data set
TLCbases[TLCbases["LICENSEE NUMBER"].str.contains('B02598')]

,index,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE


In [127]:
TLCbases.dropna(axis=0,inplace=True)

In [128]:
TLCbases = TLCbases[TLCbases['ALTERNATE NAME OF LICENSEE'].str.contains('UBER')]

In [129]:
TLCbases

,index,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE
314,314,B02395,ABATAR LLC,UBER-ABATAR LLC
317,317,B02404,"KUCHEN,LLC",UBER-KUCHEN LLC
504,95,B02617,WEITER LLC,UBER-WEITER LLC
509,100,B02682,SCHMECKEN LLC,UBER-SCHMECKEN LLC
524,115,B02764,"DANACH-NY,LLC","UBER-DANACH-NY,LLC"
525,116,B02765,GRUN LLC,UBER-GRUN LLC
554,145,B02835,DREIST NY LLC,UBER-DREIST NY LLC
555,146,B02836,DRINNEN-NY LLC,UBER-DRINNEN-NY LLC
569,160,B02864,"SIEBEN-NY,LLC","UBER-SIEBEN-NY,LLC"
570,161,B02865,"VIER-NY,LLC",UBER-VIER-NY LLC


# Read in standard FHV taxi data from TLC and compare with Uber Data

In [189]:
# Read Data
FHVdataJan15 = pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv")

In [190]:
# Visualise Data
FHVdataJan15.tail()

,Dispatching_base_num,Pickup_date,locationID
2746028,B02765,2015-01-31 23:59:02,169.0
2746029,B02765,2015-01-31 23:59:07,80.0
2746030,B02765,2015-01-31 23:59:34,186.0
2746031,B02765,2015-01-31 23:59:40,181.0
2746032,B02765,2015-01-31 23:59:48,79.0


In [193]:
# Check if missing Taxi Base exists in data set
FHVdataJan15[FHVdataJan15["Dispatching_base_num"].str.contains('B02598')].count()

Dispatching_base_num    242128
Pickup_date             242128
locationID              242128
dtype: int64

In [157]:
# Merge Taxi Bases Data with dataframe to find out uber taxis
FHVdataJan15 = FHVdataJan15.merge(TLCbases, left_on="Dispatching_base_num", right_on="LICENSEE NUMBER")

In [158]:
FHVdataJan15

,Dispatching_base_num,Pickup_date,locationID,index,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE
0,B02512,2015-01-01 00:01:21,113.0,96,B02512,UNTER LLC,UBER-UNTER LLC
1,B02512,2015-01-01 00:02:40,144.0,96,B02512,UNTER LLC,UBER-UNTER LLC
2,B02512,2015-01-01 00:06:05,141.0,96,B02512,UNTER LLC,UBER-UNTER LLC
3,B02512,2015-01-01 00:06:17,79.0,96,B02512,UNTER LLC,UBER-UNTER LLC
4,B02512,2015-01-01 00:06:23,246.0,96,B02512,UNTER LLC,UBER-UNTER LLC
5,B02512,2015-01-01 00:07:34,17.0,96,B02512,UNTER LLC,UBER-UNTER LLC
6,B02512,2015-01-01 00:07:36,188.0,96,B02512,UNTER LLC,UBER-UNTER LLC
7,B02512,2015-01-01 00:07:44,33.0,96,B02512,UNTER LLC,UBER-UNTER LLC
8,B02512,2015-01-01 00:09:07,100.0,96,B02512,UNTER LLC,UBER-UNTER LLC
9,B02512,2015-01-01 00:09:13,87.0,96,B02512,UNTER LLC,UBER-UNTER LLC


In [159]:
FHVdataJan15.dropna(axis=0,inplace=True)
FHVdataJan15.sort(columns="Pickup_date", ascending=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,Dispatching_base_num,Pickup_date,locationID,index,LICENSEE NUMBER,NAME OF LICENSEE,ALTERNATE NAME OF LICENSEE
670102,B02764,2015-01-01 00:00:05,255.0,115,B02764,"DANACH-NY,LLC","UBER-DANACH-NY,LLC"
382236,B02682,2015-01-01 00:00:08,113.0,100,B02682,SCHMECKEN LLC,UBER-SCHMECKEN LLC
382237,B02682,2015-01-01 00:00:11,234.0,100,B02682,SCHMECKEN LLC,UBER-SCHMECKEN LLC
670103,B02764,2015-01-01 00:00:13,232.0,115,B02764,"DANACH-NY,LLC","UBER-DANACH-NY,LLC"
42315,B02617,2015-01-01 00:00:14,90.0,95,B02617,WEITER LLC,UBER-WEITER LLC
42316,B02617,2015-01-01 00:00:16,225.0,95,B02617,WEITER LLC,UBER-WEITER LLC
670104,B02764,2015-01-01 00:00:19,181.0,115,B02764,"DANACH-NY,LLC","UBER-DANACH-NY,LLC"
42317,B02617,2015-01-01 00:00:27,33.0,95,B02617,WEITER LLC,UBER-WEITER LLC
670105,B02764,2015-01-01 00:00:27,231.0,115,B02764,"DANACH-NY,LLC","UBER-DANACH-NY,LLC"
670106,B02764,2015-01-01 00:00:28,68.0,115,B02764,"DANACH-NY,LLC","UBER-DANACH-NY,LLC"


In [147]:
UberJanJun15["Pickup_date"] = pd.to_datetime(UberJanJun15["Pickup_date"])

In [152]:
UberJan15 = UberJanJun15[UberJanJun15["Pickup_date"].dt.month == 1 ]
UberJan15.head()

In [154]:
UberJan15.dropna(axis=0,inplace=True)
UberJan15.sort(columns="Pickup_date", ascending=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
90608,B02598,2015-01-01 00:00:05,B02598,144
832778,B02764,2015-01-01 00:00:05,B02764,255
9371715,B02682,2015-01-01 00:00:08,B02682,113
9368311,B02682,2015-01-01 00:00:11,B02617,234
1084752,B02764,2015-01-01 00:00:13,B02764,232
7080479,B02617,2015-01-01 00:00:14,B02617,90
7077415,B02617,2015-01-01 00:00:16,B02617,225
7696272,B02764,2015-01-01 00:00:19,B02764,181
7115336,B02617,2015-01-01 00:00:27,B02764,33
11691704,B02764,2015-01-01 00:00:27,B02764,231


In [153]:
UberJan15

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
336,B02598,2015-01-18 15:44:45,NaN,243
337,B02598,2015-01-18 05:13:29,B02598,107
338,B02598,2015-01-18 15:36:13,B02598,170
339,B02598,2015-01-18 00:02:54,NaN,144
340,B02598,2015-01-18 17:55:53,B02598,211
341,B02598,2015-01-18 01:09:42,B02764,263
342,B02598,2015-01-18 11:06:58,B02598,7
343,B02598,2015-01-18 22:48:30,B02598,45
344,B02598,2015-01-18 00:55:44,B02598,33
345,B02598,2015-01-18 02:28:16,B02598,125


In [160]:
print(FHVdataJan15.Dispatching_base_num.unique())
print(UberJan15.Dispatching_base_num.unique())

['B02512' 'B02617' 'B02682' 'B02764' 'B02765']
['B02598' 'B02764' 'B02682' 'B02617' 'B02512' 'B02765']


In [ ]:
common_cols = .columns.tolist()                         #generate list of column names
df12 = pd.merge(df1, df2, on=common_cols, how='inner')     #extract common rows with merge
df2 = df2[~df2['A'].isin(df12['A'])]


In [176]:
UberJan15[UberJan15["Dispatching_base_num"].str.contains('B02598')].count()
#TLCbases[TLCbases["LICENSEE NUMBER"].str.contains('B02598')]

Dispatching_base_num    242128
Pickup_date             242128
Affiliated_base_num     242128
locationID              242128
dtype: int64

In [178]:
len(UberJan15)-len(FHVdataJan15)

242128